In [ ]:
!pip install underthesea

In [ ]:
import tensorflow as tf
from tensorflow import keras
import re
import numpy as np
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras import layers
from keras.layers import *
from keras.models import *
from underthesea import word_tokenize
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import pandas as pd
import string

# **preprocess_data**

In [ ]:
url = "haha.xlsx"

# Đọc file Excel, giả sử tên file là 'du_lieu.xlsx' và dữ liệu ở sheet đầu tiên
df = pd.read_excel(url)

# Giả sử các cột bạn muốn kiểm tra là 'Cột 1' và 'Cột 2'
col1 = 'sentence'
col2 = 'label'

def is_punctuation(value):
    return all(char in string.punctuation for char in str(value))

# Loại bỏ hàng nếu một trong hai cột trống hoặc có chứa chỉ dấu chấm câu
df = df.dropna(subset=[col1, col2])  # Loại bỏ hàng nếu có cột nào đó bị trống
df = df[~df[col1].apply(is_punctuation) & ~df[col2].apply(is_punctuation)]

In [ ]:
all_data = df[col1]
label = df[col2]
label3 = [0 if i == 'positive' else 1 for i in label]
print(all_data)
print(len(label3))

0       Chào điện máy Xanh tôi có mua con s23 Ultra bê...
1       Máy tốt, không lỗi gì nhé. Camera và bộ công c...
2       Mới mua 22tr9 dc mấy hôm nay giảm còn 20tr9 ch...
3                                         May muot lam ok
4       Hài lòng về sản phẩm. Rất cảm ơn các bạn nhân ...
                              ...                        
2917                                                  Tốt
2918                                  Loa không nghe được
2919    Bình thường. Loa nhỏ nghe k rõ, k xem được dan...
2920    Loa trong mà tích hợp ở mặt lưng như nokia 128...
2921                        Không được tốt tắt nguồn hoài
Name: sentence, Length: 2911, dtype: object
2911


In [ ]:
path_positive = "positive.txt"
path_negative = "negative.txt"
dt_positive = []
dt_negative = []
with open(path_positive, 'r') as file_1:
  for line in file_1:
    dt_positive.append(line.strip())

with open(path_negative, 'r') as file_2:
  for line in file_2:
    dt_negative.append(line.strip())

label_1 = [0 for i in range(len(dt_positive))]
label_2 = [1 for i in range(len(dt_negative))]

In [ ]:
dt = np.concatenate([dt_positive, dt_negative, all_data], axis=-1)
labels = np.concatenate([label_1, label_2, label3], axis=-1)

In [ ]:
def tokenize(text):
  text = re.sub(r'[^\w\s]', '', text).lower()
  text = word_tokenize(text)
  return text
def w2id_id2w(word,):
  w2id = {}
  id2w = {}
  for id, w in enumerate(word):
    w2id[w] = id + 2
    id2w[id] = w
  w2id['pad'] = 1
  id2w[1] = 'pad'
  return w2id, id2w

def pad_sequen(data, w2id):
  dt = []
  for text in data:
    senten = []
    for word in text:
      senten.append(w2id[word])
    dt.append(senten)
  dt = pad_sequences(dt, padding='post')
  return dt



In [ ]:
words = set()
data_all = []
for text in dt:
    text = tokenize(text)
    for w in text:
      words.add(w)
    data_all.append(text)

w2id, id2w = w2id_id2w(words,)
data_train = pad_sequen(data_all, w2id)

In [ ]:
print(len(dt))
print(len(labels))

3339
3339


# tạo dictionary

In [ ]:
import yaml

In [ ]:
file_name = "/content/drive/MyDrive/Nlp/dictionary_yaml.yaml"

# Viết dictionary vào file YAML
with open(file_name, "w") as file:
    yaml.dump(w2id, file)

# convert data to tensor

In [ ]:
maxlen = data_train.shape[1]
print(maxlen)

307


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((data_train, labels))
train_dataset = train_dataset.shuffle(buffer_size=maxlen)
train_dataset = train_dataset.batch(batch_size=32)

In [ ]:
print(train_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 307), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


build_model

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionembedding(layers.Layer):
  def __init__(self, maxlen, vocab_size, embed_dim):
    super().__init__()
    self.token_embed = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
    self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

  def call(self, x):
    maxlen = np.shape(x)[-1]
    positions = tf.range(start=0, limit=maxlen, delta=1)
    positions = self.pos_emb(positions)
    x = self.token_embed(x)
    return x + positions

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32
inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionembedding(maxlen, len(words) + 2, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer=Adam(0.0005), loss=sparse_categorical_crossentropy, metrics=['accuracy'])
model.fit(train_dataset, epochs=20, verbose=1)

Epoch 1/20
105/105 [==============================] - 22s 134ms/step - loss: 0.5472 - accuracy: 0.7616
Epoch 2/20
105/105 [==============================] - 11s 103ms/step - loss: 0.5333 - accuracy: 0.7562
Epoch 3/20
105/105 [==============================] - 8s 74ms/step - loss: 0.5074 - accuracy: 0.7706
Epoch 4/20
105/105 [==============================] - 5s 46ms/step - loss: 0.5066 - accuracy: 0.7715
Epoch 5/20
105/105 [==============================] - 5s 53ms/step - loss: 0.4234 - accuracy: 0.7901
Epoch 6/20
105/105 [==============================] - 3s 24ms/step - loss: 0.3107 - accuracy: 0.8577
Epoch 7/20
105/105 [==============================] - 3s 29ms/step - loss: 0.2562 - accuracy: 0.8925
Epoch 8/20
105/105 [==============================] - 2s 20ms/step - loss: 0.2222 - accuracy: 0.9143
Epoch 9/20
105/105 [==============================] - 3s 28ms/step - loss: 0.1904 - accuracy: 0.9290
Epoch 10/20
105/105 [==============================] - 2s 24ms/step - loss: 0.1733 - ac

In [ ]:
model.save("/content/drive/MyDrive/Nlp/model_comment.tf")

In [ ]:
model_comment = keras.models.load_model("/content/drive/MyDrive/Nlp/model_comment.tf")

In [ ]:
def proces_input(data, word_to_id, maxlen):
  data = word_tokenize(re.sub(r'[^\w\s]', '', data).lower())
  inputs = [word_to_id[word] if word in word_to_id.keys() else 1 for word in data]
  inputs = pad_sequences([inputs], maxlen=maxlen,padding='post',truncating='post')
  return inputs

dic1 = {1: 'positive', 0: 'negative'}
def pre(model, text, d, loaded_data):

  inputs = proces_input(text,loaded_data,307)
  predic = model(inputs)
  predic = np.argmax(predic, axis=-1)
  return d[predic[0]]


In [ ]:
input = "điện thoại như cục cứt đừng nên mua nha mọi người"
y_pre = pre(model_comment, input, dic1, w2id)
print(y_pre)

negative
